# Mexican Senate Data

## Importing necessary libraries

In [1]:
# importing required modules
import requests
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup
import numpy as np
from lxml import etree
import re
from selenium import webdriver
import time

## Senator Database, Exports to CSV in data folder.

### Importing senator table

In [71]:
def get_senators():
    senators_url = 'https://www.senado.gob.mx/65/datosAbiertos/senadoresDatosAb.json'
    senators_json = requests.get(senators_url).json()
    senators = pd.DataFrame.from_dict(senators_json)
    senators = senators.rename(columns={"idSenador": "senator_id"})
    return senators

In [72]:
senators = get_senators()

In [73]:
#Creating a field that includes first and last names to join with initiatives+proposals table.
senators["senadores"] = senators["Nombre"].str.strip()+" "+senators["Apellidos"].str.strip()

### Importing attendance data and adding to senator table

In [5]:
def get_senator_attendance():
    
    senators = get_senators()
    
    senator_ids = senators["senator_id"].tolist()
    
    senator_attendance = pd.DataFrame()
    senator_attendance["senator_id"] = ""
    senator_attendance["session_date"] = ""
    senator_attendance["attendance_record"] = ""

    counter = 0
    for sen in senator_ids:
        url = f'https://www.senado.gob.mx/65/asistencias/{sen}#info'
        html = requests.get(url)
        content = BeautifulSoup(html.text, 'html.parser')
        content_x = etree.HTML(str(content))
        dates = content_x.xpath('//*[@id="imPage"]/div[7]/div[2]/div/div[2]/section/div/div/table/tbody//a')
        att_records = content_x.xpath('//*[@id="imPage"]/div[7]/div[2]/div/div[2]/section/div/div/table/tbody//strong')
        for i in range(len(dates)):
            senator_attendance.at[i+counter, 'senator_id'] = sen
            senator_attendance.at[i+counter, 'session_date'] = dates[i].text
            senator_attendance.at[i+counter, 'attendance_record'] = att_records[i].text
        counter += len(dates)

    senator_attendance["attendance_score"] = senator_attendance["attendance_record"].copy()
    senator_attendance["attendance_score"] = senator_attendance["attendance_score"].map(lambda x: 1 if x == "Asistencia" else 0)
    senator_attendance = pd.merge(senator_attendance, senators[['senator_id','Fraccion', 'Estado', 'Apellidos', 'Nombre', 'tipoEleccion']], on='senator_id', how='left')

    senator_attendance["full_name"] = senator_attendance['Nombre'] + " " + senator_attendance['Apellidos']
    
    senator_attendance = senator_attendance.groupby(['senator_id', 'full_name', 'Fraccion', 'Estado', 'tipoEleccion'], as_index=False)[['attendance_score']].mean()

    return senator_attendance

In [6]:
senator_attendance = get_senator_attendance()

In [74]:
senators = senators.merge(senator_attendance[["senator_id", "attendance_score"]], how="left", on="senator_id")

### Importing initiatives and proposals, concatenating both and adding senator ids

In [8]:
def get_initiatives():
    """fucntion that extracts initiatives from Senate JSON."""
    
    init_64_url = 'https://www.senado.gob.mx/65/datosAbiertos/iniciativa_64.json'
    init_65_url = 'https://www.senado.gob.mx/65/datosAbiertos/iniciativa_65.json'
    
    init_64_json = requests.get(init_64_url).json()
    init_65_json = requests.get(init_65_url).json()
    
    init_64 = pd.DataFrame.from_dict(init_64_json)
    init_65 = pd.DataFrame.from_dict(init_65_json)
    
    initiatives = pd.concat([init_64, init_65])
    
    initiatives['fecha_presentacion'] = pd.to_datetime(initiatives['fecha_presentacion'],errors='coerce')
    initiatives['fecha_aprobacion'] = pd.to_datetime(initiatives['fecha_aprobacion'],errors='coerce')
    
    initiatives = initiatives.set_index('id')
        
    return initiatives

In [9]:
def get_proposals():
    """fucntion that extracts proposals from Senate JSON."""
    
    prop_64_url = 'https://www.senado.gob.mx/65/datosAbiertos/proposicion_64.json'
    prop_65_url = 'https://www.senado.gob.mx/65/datosAbiertos/proposicion_65.json'
    
    prop_64_json = requests.get(prop_64_url).json()
    prop_65_json = requests.get(prop_65_url).json()
    
    prop_64 = pd.DataFrame.from_dict(prop_64_json)
    prop_65 = pd.DataFrame.from_dict(prop_65_json)
    
    proposals = pd.concat([prop_64, prop_65])
    
    proposals['fecha_presentacion'] = pd.to_datetime(proposals['fecha_presentacion'],errors='coerce')
    proposals['fecha_aprobacion'] = pd.to_datetime(proposals['fecha_aprobacion'],errors='coerce')
    
    proposals = proposals.set_index('id')
    
    return proposals

In [157]:
#Create concatenated df that includes initiatives and proposals.
initiatives = get_initiatives()
proposals = get_proposals()
inipros = pd.concat([initiatives, proposals])

In [131]:
print(f"Inipros df has {inipros.shape[0]} initiatives with {inipros.shape[1]} features.")

Inipros df has 9396 initiatives with 13 features.


In [158]:
#creates a 1:1 relationship between initiative/proposal and senator (in case where more than 1 senator proposes).
inipros["senadores"] = inipros["senadores"].apply(lambda x:x.strip().split("<br>"))

for i, row in inipros.iterrows():
    senator_ids = []
    for senator in row["senadores"]:
        strt_pos = senator.find('(')
        senator = senator[:strt_pos-1].strip()
        senator_ids.append(senator)
    inipros.at[i, "senadores"] = senator_ids[:-1]

inipros = inipros.explode("senadores")

In [159]:
#Manually change names in inipros so they match senator names from senator table.

inipros.loc[inipros["senadores"] == "Geovanna del Carmen Bañuelos de La Torre", "senadores"] = "Geovanna Bañuelos"
inipros.loc[inipros["senadores"] == "Noé Fernando Castañón Ramírez", "senadores"] = "Noé Castañón"
inipros.loc[inipros["senadores"] == "José Clemente Castañeda Hoeflich", "senadores"] = "Clemente Castañeda Hoeflich"
inipros.loc[inipros["senadores"] == "Juan Manuel Zepeda Hernández", "senadores"] = "Juan Zepeda"
inipros.loc[inipros["senadores"] == "Patricia Mercado Castro", "senadores"] = "Patricia Mercado"
inipros.loc[inipros["senadores"] == "Dante Delgado Rannauro", "senadores"] = "Dante Delgado"
inipros.loc[inipros["senadores"] == "Bertha Xóchitl Gálvez Ruiz", "senadores"] = "Xóchitl Gálvez Ruiz"
inipros.loc[inipros["senadores"] == "Lilly Téllez García", "senadores"] = "Lilly Téllez"
inipros.loc[inipros["senadores"] == "Raúl Bolaños Cacho Cué", "senadores"] = "Raúl Bolaños-Cacho Cué"
inipros.loc[inipros["senadores"] == "Elvia Marcela Mora Arellano", "senadores"] = "Marcela Mora"
inipros.loc[inipros["senadores"] == "Minerva Citlalli Hernández Mora", "senadores"] = "M. Citlalli Hernández Mora"
        
inipros[inipros["titulo"]=='Proyecto de decreto por el que se expide la Ley General de Aguas y se abroga la Ley de Aguas Nacionales.']

,titulo,fecha_presentacion,sentido_dictamen,url_gaceta,estado,sintesis,legislatura,senadores,comisiones,leyes_modifica,tipo,camara_origen,fecha_aprobacion
id,,,,,,,,,,,,,
736882,Proyecto de decreto por el que se expide la Le...,2020-03-03,No Aplica,104613,Pendiente,Propone que se abrogue la Ley de Aguas Naciona...,64,Lucía Virginia Meza Guzmán,Recursos Hidráulicos (Coordinadora) <br>Estudi...,Ley de Aguas Nacionales<br>Expedición de Nueva...,iniciativa,No Aplica,NaT
736882,Proyecto de decreto por el que se expide la Le...,2020-03-03,No Aplica,104613,Pendiente,Propone que se abrogue la Ley de Aguas Naciona...,64,Daniel Gutiérrez Castorena,Recursos Hidráulicos (Coordinadora) <br>Estudi...,Ley de Aguas Nacionales<br>Expedición de Nueva...,iniciativa,No Aplica,NaT
736882,Proyecto de decreto por el que se expide la Le...,2020-03-03,No Aplica,104613,Pendiente,Propone que se abrogue la Ley de Aguas Naciona...,64,Aníbal Ostoa Ortega,Recursos Hidráulicos (Coordinadora) <br>Estudi...,Ley de Aguas Nacionales<br>Expedición de Nueva...,iniciativa,No Aplica,NaT
736882,Proyecto de decreto por el que se expide la Le...,2020-03-03,No Aplica,104613,Pendiente,Propone que se abrogue la Ley de Aguas Naciona...,64,Rocío Adriana Abreu Artiñano,Recursos Hidráulicos (Coordinadora) <br>Estudi...,Ley de Aguas Nacionales<br>Expedición de Nueva...,iniciativa,No Aplica,NaT
736882,Proyecto de decreto por el que se expide la Le...,2020-03-03,No Aplica,104613,Pendiente,Propone que se abrogue la Ley de Aguas Naciona...,64,Cruz Pérez Cuellar,Recursos Hidráulicos (Coordinadora) <br>Estudi...,Ley de Aguas Nacionales<br>Expedición de Nueva...,iniciativa,No Aplica,NaT
736882,Proyecto de decreto por el que se expide la Le...,2020-03-03,No Aplica,104613,Pendiente,Propone que se abrogue la Ley de Aguas Naciona...,64,Martí Batres Guadarrama,Recursos Hidráulicos (Coordinadora) <br>Estudi...,Ley de Aguas Nacionales<br>Expedición de Nueva...,iniciativa,No Aplica,NaT
736882,Proyecto de decreto por el que se expide la Le...,2020-03-03,No Aplica,104613,Pendiente,Propone que se abrogue la Ley de Aguas Naciona...,64,M. Citlalli Hernández Mora,Recursos Hidráulicos (Coordinadora) <br>Estudi...,Ley de Aguas Nacionales<br>Expedición de Nueva...,iniciativa,No Aplica,NaT
736882,Proyecto de decreto por el que se expide la Le...,2020-03-03,No Aplica,104613,Pendiente,Propone que se abrogue la Ley de Aguas Naciona...,64,Eva Eugenia Galaz Caletti,Recursos Hidráulicos (Coordinadora) <br>Estudi...,Ley de Aguas Nacionales<br>Expedición de Nueva...,iniciativa,No Aplica,NaT
736882,Proyecto de decreto por el que se expide la Le...,2020-03-03,No Aplica,104613,Pendiente,Propone que se abrogue la Ley de Aguas Naciona...,64,Gricelda Valencia de la Mora,Recursos Hidráulicos (Coordinadora) <br>Estudi...,Ley de Aguas Nacionales<br>Expedición de Nueva...,iniciativa,No Aplica,NaT


In [160]:
#Inner join on senator names to ensure only initiatives that match senator ids from table remain.
inipros = inipros.merge(senators[["senadores", "senator_id"]], how='left', on='senadores')

In [161]:
print(f"Inipros df has {inipros.shape[0]} initiatives with {inipros.shape[1]} features.")

Inipros df has 12828 initiatives with 14 features.


In [164]:
#Remove initiatives that do not match with senator names.
inipros = inipros[inipros["senator_id"].notnull()]

In [165]:
print(f"Inipros df has {inipros.shape[0]} initiatives with {inipros.shape[1]} features.")

Inipros df has 9014 initiatives with 14 features.


### Add list of initiative strings back to senator table

In [167]:
senators["initiative_list"] = ""

In [168]:
#Function that creates a list of initiative syntheses and then adds to senator database.
for i, row in senators.iterrows():
    initiatives = []
    relevant_inipros = inipros[inipros["senator_id"] == str(row["senator_id"])]["sintesis"]
    [initiatives.append(initiative.replace('\r\n\r\n', ' ')) for initiative in relevant_inipros]
    senators.at[i, "initiative_list"] = initiatives

In [169]:
#Creates dummy summary of a all initiatives, to be replaced by BERT or BETO summaries.
senators["initiatives_summary_dummy"] = senators["initiative_list"].apply(lambda x: "".join(x))

### Export file to CSV in data folder

In [170]:
senators.to_csv(r'/Users/jmlunamugica/code/jomilu93/sivico/senators_data.csv')

# Classifying initiatives & proposals into topics with LDA (DEPRECATED)

## LDA Approach

### Preprocessing text

In [244]:
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [257]:
def clean(column):
    """Remove punctuation, make strings lower case, remove numbers. Tokenize, remove stopwords and lemmatize."""
    #Removing punctuation.
    for punctuation in string.punctuation:
        column = column.apply(lambda x: x.replace(punctuation, ''))
    #Making lower case and removing whitespace.
    column = column.apply(lambda x: x.lower().strip())
    #Removing numbers
    column = column.apply(lambda x: re.sub(r'[0-9]', '', x))
    #Tokenize all rows.
    column = column.apply(lambda x: word_tokenize(x))
    #Remove stopwords and words too frequently present in initiative language.
    stop_words = set(stopwords.words('spanish'))
    stop_words_extra = ("exhorta", "modificar", "actualizar", "política", "general", "caso", "derecho", "materia", "virtud", "referencias", "cambiar", "deberán", "día", "año", "denominación", "distrito", "cámara", "senadores", "normativa", "senado", "objetivo", "cumplimiento", "ordenamiento", "república", "reforma", "cada", "dar", "federal", "secretaría", "mención", "paso", "dejar", "principio", "ser", "paridad", "así", "derechos", "reformar", "propone", "nacional", "establecer", "méxico", "persona", "ley", "ciudad", "deberá", "legal", "personas")
    column = column.apply(lambda x: [w for w in x if w not in stop_words])
    column = column.apply(lambda x: [w for w in x if w not in stop_words_extra])
    # Lemmatizing the verbs
    column = column.apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos = "v") for word in x])
    # 2 - Lemmatizing the nouns
    column = column.apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos = "n") for word in x])
    # Rejoin words to make sentences
    column = column.apply(lambda x: " ".join(x))
    return column

In [258]:
inipros["sintesis_clean"] = clean(inipros["sintesis"])

### Training vectorization model

In [252]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [259]:
vectorizer = TfidfVectorizer()

vectorized_text = vectorizer.fit_transform(inipros["sintesis_clean"])

# Instantiate the LDA 
n_components = 15
lda_model = LatentDirichletAllocation(n_components=n_components)

# Fit the LDA on the vectorized documents
lda_model.fit(vectorized_text)

LatentDirichletAllocation(n_components=15)

### Visualize potential topics

In [255]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i]) for i in np.argsort(topic)[:-5 -1:-1]])

In [260]:
print_topics(lda_model, vectorizer)

Topic 0:
[('cuidados', 8.152377824370088), ('plazo', 6.54867271043552), ('moratorios', 6.251462236713305), ('tratándose', 6.140648831027995), ('circuito', 5.822456621174861)]
Topic 1:
[('prisión', 17.33962346795972), ('delitos', 15.172324267906053), ('delito', 15.069667077986509), ('votos', 14.898576003258666), ('democrática', 14.693022763766855)]
Topic 2:
[('inclusivo', 18.261246187847167), ('lenguaje', 17.418572834911885), ('constitucionales', 16.878337552432512), ('disposiciones', 15.727013331203173), ('incorporar', 14.849857208397227)]
Topic 3:
[('órganos', 17.763659837930643), ('federales', 15.273162946841973), ('uso', 11.773341107108418), ('tribunales', 10.711978953185039), ('administrativos', 10.623157522217339)]
Topic 4:
[('indígenas', 34.68765761725618), ('pueblo', 30.01337346202277), ('agua', 27.900267103930567), ('ordenamientos', 27.526534543323102), ('inclusión', 25.80922310283219)]
Topic 5:
[('género', 90.26950785407557), ('garantizar', 76.29735575292742), ('salud', 15.686

### Test with real initiatives

In [267]:
random_num = np.random.randint(0, len(inipros))
example = [inipros["sintesis"][random_num]]
example_df = pd.DataFrame(example, columns = ["text"])
print(example_df["text"][0])

ÚNICO. El Senado de la República exhorta respetuosamente la Secretaria de Comunicaciones y Transportes, para que, en el marco de sus atribuciones, revise las tarifas que se cobran en las carreteras de cuota, con la finalidad de evitar cobros excesivos que afecten la economía de los usuarios, y se mejore y mantenga el estado físico de la red carretera del país.


In [268]:
clean_example = clean(example_df["text"])
example_vectorized = vectorizer.transform(clean_example)
lda_vectors = lda_model.transform(example_vectorized)
lda_vectors

array([[0.01157094, 0.01157095, 0.01157098, 0.01157095, 0.01157094,
        0.01157095, 0.30414982, 0.01157098, 0.01157097, 0.01157095,
        0.01157095, 0.01157097, 0.01157104, 0.01157097, 0.54542765]])